In [1]:
import torch
import pandas as pd
import plotnine as pn
import sys
# sys.path.append("/Users/nathanielimel/uci/projects/ibsg/src")

In [2]:
curve_data = pd.read_csv("/Users/nathanielimel/uci/projects/ibsg/multirun/states=100/signals=100/prior=-8/dist=squared_dist/meaning_certainty=0/curve_points.csv")

# maps gamma to H(U|M) bits
gamma_dict = {
    3: 0.0,
    0: 1.532,
    -1: 3.163,
    -2: 4.726,
    # -3: 6.078,
}
sim_data = {
    key: pd.read_csv(f"noise={key}.csv") for key in gamma_dict
}

In [3]:
# fill aesthetics
for key, val in gamma_dict.items():
    sim_data[key]["noise"] = str(val) # categorical

if list(sim_data):
    sim_data = pd.concat(list(sim_data.values()))

In [4]:
# this is a dirty and beautiful hack from
# https://waldyrious.net/viridis-palette-generator/
fill_values_dict = {
    str(k): v for k, v in {
        0.0: "#440154",
        1.532: "#31688e",
        3.163: "#35b779",
        4.726: "#fde725", 
    }.items() }

if True:
    fill_values_dict = {k: "light-blue" for k,v in fill_values_dict.items()}

In [5]:
plot = (
    # Set data and the axes
    pn.ggplot(
        data=curve_data, mapping=pn.aes(x="complexity", y="accuracy")
    )  
    + pn.geom_line(size=3) # IB curve
    # + pn.geom_point(size=1)
    + pn.geom_ribbon(
    mapping=pn.aes(
        x="complexity",
        ymin="accuracy",
        ymax=torch.inf,
        ),
        fill="gray",
        alpha=0.3,
    )
)

if True:
    plot = (
        plot 
        + pn.geom_jitter( # emergent langs
            data=sim_data,
            mapping=pn.aes(
                # fill="discr_need",
                fill="noise",
                # shape="language",
                ),
            alpha=0.3,
            size=5,
        )
        + pn.scale_fill_manual(values = fill_values_dict)
    )
    
plot = (
    plot
    + pn.xlab("Complexity, $I(M;W)$ bits")
    + pn.ylab("Accuracy, $I(W;U)$ bits")

    + pn.theme_classic()
    + pn.theme(
        # Axis font
        axis_title=pn.element_text(size=24),
        axis_text=pn.element_text(size=12),
        # Legend position
        # legend_position=(0.75, 0.4),
        legend_position='none',
    )    
)

print(plot)
plot.save("temp_figure", width=10, height=10, dpi=300,)

ValueError: Invalid RGBA argument: 'light-blue'